In [4]:
# Import Necessary Libraries

from dotenv import load_dotenv, dotenv_values
import google.generativeai as genai
from IPython.display import Markdown, display
import os

# Load .env file

load_dotenv()

# Set the TESSDATA_PREFIX environment variable

tessdata = os.getenv("TESSDATA_PREFIX")
os.environ['TESSDATA_PREFIX'] = tessdata

# Set the Google api key

my_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=my_api_key)

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings,GoogleGenerativeAI

llm = GoogleGenerativeAI(model ="gemini-pro",temperature=0)
embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") 
model = ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)

In [6]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["oracle_sql", "mysql", "mongodb"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
structured_llm = model.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
chain = prompt | structured_llm




In [7]:
from langchain_core.prompts import PromptTemplate

oracle_chain = PromptTemplate.from_template(
    """You are an expert in oracle database. \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)
mysql_chain = PromptTemplate.from_template(
    """You are an expert in mysql database. \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)
mongodb_chain = PromptTemplate.from_template(
    """You are an expert in mongodb. \
Respond to the following question:
Question: {question}
Answer:"""
) | ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)

In [8]:
def choose_route(question):
    result = chain.invoke({"question": question})
    if "oracle_sql" in result.datasource.lower():
        ### Logic here 
        print("Using Oracle")
        return oracle_chain
    elif "mysql" in result.datasource.lower():
        ### Logic here 
        print("Using mysql")
        return mysql_chain
    else:
        ### Logic here 
        print("Using mongodb")
        return mongodb_chain



#full_chain = router | RunnableLambda(choose_route)

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    choose_route) 

In [15]:
final_chain =  (
    {"query": RunnablePassthrough()}
    | full_chain
    | llm
    | StrOutputParser()
)

In [20]:
%%time

result = full_chain.invoke({"question": "Write a stored procedure to concatenate first name and last name in mysql"})
print(result.content)

Using mysql
```sql
DELIMITER //

CREATE PROCEDURE ConcatenateNames(IN userId INT, OUT fullName VARCHAR(255))
BEGIN
  SELECT CONCAT(firstName, ' ', lastName) INTO fullName
  FROM users
  WHERE id = userId;
END //

DELIMITER ;
```

**Explanation:**

1. **DELIMITER //:** This line changes the default delimiter from `;` to `//` to allow the stored procedure definition to span multiple lines.
2. **CREATE PROCEDURE ConcatenateNames(IN userId INT, OUT fullName VARCHAR(255)):** This line defines the stored procedure named `ConcatenateNames`. It takes an input parameter `userId` of type `INT` and an output parameter `fullName` of type `VARCHAR(255)`.
3. **SELECT CONCAT(firstName, ' ', lastName) INTO fullName FROM users WHERE id = userId;**: This line performs the following:
   - `CONCAT(firstName, ' ', lastName)`: Concatenates the `firstName` and `lastName` columns from the `users` table, separated by a space.
   - `INTO fullName`: Stores the concatenated result into the output parameter `fullN